## Retrieve NWM retrospective snow data (Grid-to-point) from version 2.1

* This code retrieves snow depth and SWE data data from the NWM retrospective dataset stored in AWS (https://registry.opendata.aws/nwm-archive/). 
* The values are extracted from the grid cell that matches a lat/lon location.

In [ ]:
import os
import pandas as pd
import xarray
import numpy
import pyproj
import matplotlib.pyplot as plt
from datetime import datetime
import s3fs
import hvplot.xarray
import dask.array as da

from dask.distributed import Client
client = Client()
client

#### Reand NWM dataset

In [ ]:
s3_path = 's3://noaa-nwm-retrospective-2-1-zarr-pds/ldasout.zarr'

In [ ]:
# Connect to S3
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)

In [ ]:
%%time
# load the dataset
ds = xarray.open_zarr(store=store, consolidated=True)

#### Define lat/lon locations where data will be extracted

In [ ]:
# Path to where the sites of interest are located
sitesPath = './Input/'

In [ ]:
# Read sites
#-------------
sites_loc = pd.read_csv(sitesPath+'selStn_snow.csv',dtype={'siteID': 'string','name':'string','Source': 'string'})
lat = sites_loc['latitude'].values.tolist()
lon = sites_loc['longitude'].values.tolist()
siteIDs = sites_loc['siteID'].values.tolist()

In [ ]:
# define the input crs
wrf_proj = pyproj.Proj(proj='lcc',
                       lat_1=30.,
                       lat_2=60., 
                       lat_0=40.0000076293945, lon_0=-97., # Center point
                       a=6370000, b=6370000) 

# define a target coordinate system to convert locations into the projection of our NWM data
target_crs = wrf_proj

# Obs proj.
wgs_proj = pyproj.Proj(proj='latlong', datum='WGS84')

# Define transformer to reproject the station locations to the coordinates of NWM/LSM
transformer = pyproj.Transformer.from_crs(wgs_proj.crs, target_crs.crs)

In [ ]:
sites = numpy.array([siteIDs])

# Reproject to AORC/NWM coordinates
xx0, yy0 = transformer.transform(lon,lat)

xx = xarray.DataArray(xx0, coords=sites, dims=['location'])
yy = xarray.DataArray(yy0, coords=sites, dims=['location'])

#### Extract data over the full retrospective period by time chunks

In [ ]:
savePath = "./Output/"

In [ ]:
# slice all data in time chunks
start_date = datetime.strptime("1979-02-01 00:00:00", "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime("2021-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

date_list = pd.date_range(start_date, end_date, periods=11)
print(date_list)

In [ ]:
%%time
# Loop to process the data in time chunks
max_lon = ds["x"].max()
max_lat = ds["y"].max()

for i in range(len(date_list)-1):
    print("Processing block of dates from",date_list[i], "to",date_list[i+1])
    timerange = slice(str(date_list[i]), str(date_list[i+1]))
    print(timerange)
    dat = ds[['SNOWH', 'SNEQV']].sel(time=timerange,x=slice(1e6,max_lon),y=slice(0,max_lat)).persist()
    
    # Extract the values at the point locations
    values_temp = dat.sel(x=xx, y=yy, method='nearest').to_dataframe()
    values_temp.to_csv(f'{savePath}NWM_Snowretro_{str(i)}.csv')
    print("Data saved...Done")

    # Delete unnecesary data to save memory
    del(dat,values_temp)